Import Packages 

In [6]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 1.0/390.3 MB 7.2 MB/s eta 0:00:54
   ---------------------------------------- 2.9/390.3 MB 8.0 MB/s eta 0:00:49
   ---------------------------------------- 4.7/390.3 MB 7.9 MB/s eta 0:00:49
    --------------------------------------- 6.6/390.3 MB 8.1 MB/s eta 0:00:48
    --------------------------------------- 7.9/390.3 MB 7.7 MB/s eta 0:00:50
    --------------------------------------- 8.9/390.3 MB 7.3 MB/s eta 0:00:53
   - -------------------------------------- 10.2/390.3 MB 6.9 MB/s eta 0:00:55
   - -------------------------------------- 11.3/390.3 MB 6.8 MB/s eta 0:00:56
   - -------------------------------------- 12.6/390.3 MB 6.6 MB/s eta 0:00:57
   - -------------------------------------- 13.9/390.3 MB 6.5 MB/s eta 0:00:58
   - -------------------------------------- 14.9/390.3 MB 6.4 MB/s 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Ari Sinervo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
from PIL import Image
import os
import cv2
import random
import torch
from torch import nn, optim
import math
from IPython import display
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import torch.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
import pdb

Create list of morph classes

In [ ]:
classes = ()

Load dataset

Greyscale Images for contrast detection

Image size generalization

In [ ]:
def Load_Folder_Images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = Image.open(os.path.join(folder_path, filename))
        img = img.resize((64, 64))
        img = np.array(img)
        images.append(img)
    return np.array(images)

def Resize_Images(images, size):
    resized_images = []
    for img in images:
        img = Image.fromarray(img)
        img = img.resize((size, size))
        img = np.array(img)
        resized_images.append(img)
    return np.array(resized_images)

def convert_and_save_images_to_greyscale(images, path_save):
    greyscale_images = []
    if not os.path.exists(path_save):
        os.makedirs(path_save)
    for filename, img in images:
        grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        greyscale_images.append(grey_img)
        save_path = os.path.join(path_save, filename)
        cv2.imwrite(save_path, grey_img)
    return np.array(greyscale_images)

In [ ]:
def load_images(data_dir, img_size):
    images = []
    labels = []
    class_names = sorted(os.listdir(data_dir))  # Get class names from subfolders
    
    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            # Only process valid image files
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                print(f"Skipping non-image file: {img_file}")
                continue
            try:
                # Load the image, resize, and convert to grayscale
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (img_size, img_size))
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Failed to process {img_path}: {e}")
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names

# Set parameters
train_dir = "Train"  # Path to the training dataset
test_dir = "Test"    # Path to the testing dataset
img_size = 64        # Resize images to 64x64 pixels

In [ ]:
# load data
images = Load_Folder_Images(path)

In [ ]:
greys = convert_images_to_greyscale(images, path_save)

Isolate gecko from background

we need to isolate the gecko from the background first to generalize how the model will be seeing the images later on, we do not want the morphology to be affected by how the background looks. So, a quick review of sights like morph market show that there are three main conditions we need to deal with, a white background, a black background, and mst commonly a background involving natural elements like grass or woodchips. 

In [ ]:
class Gecko_Classifier(nn.module):
    def __init__(self):
        super(Gecko_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=2)
        self.pool - nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.15)
        self.dropout2 = nn.Dropout(0.25)
        self.dropout3 = nn.Dropout(0.4)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout2(x)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout3(x)
        x = nn.functional.softmax(self.fc2(x))
        return x
    
class Gecko_lineage(nn.module):
    def __initi__(self):
        
        pass
    def forward(self, x)
        return x
    
def train_model():
    pass

In [ ]:
# Load training and testing data
X_train, y_train, class_names = load_images(train_dir, img_size)
X_test, y_test, _ = load_images(test_dir, img_size)  # Use the same class names

# Normalize images
X_train = X_train / 255.0  # Normalize pixel values to [0, 1]
X_test = X_test / 255.0

# Add a channel dimension for grayscale images
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# 2. Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),  # Randomly flip images horizontally
    layers.RandomRotation(0.05),      # Rotate images by ±10%
    layers.RandomZoom(0.05)           # Randomly zoom into the image
])

Greyscale Morphology

In [ ]:
# run on single morph data set
# run on multi morph data set